In [1]:
import pandas as pd
import numpy as np
import sqlite3
import importlib

# keeping company information in additional file
import data_file

In [2]:
importlib.reload(data_file)

<module 'data_file' from 'C:\\Users\\30050122\\Documents\\python\\data_preps\\prj_23012_order_intake_file_update\\data_file.py'>

In [3]:
targets = pd.read_excel('data_files/targets.xlsx')
targets['tm_check'] = 'no'
targets['pure tm'] = 'no'

In [4]:
targets_fy21 = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY21', 'tm_check', 'pure tm']]
targets_fy21 = targets_fy21.dropna(subset=['FY21'])
targets_fy21 = targets_fy21[targets_fy21['FY21'] != 0]
targets_fy21['FY21'] = targets_fy21['FY21'] / 12
targets_fy21['FY'] = 2021
targets_fy21 = targets_fy21.rename(columns={'FY21': 'target_order_intake_amount_eur'})
num_fy21 = len(targets_fy21)

targets_fy22 = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY22', 'tm_check', 'pure tm']]
targets_fy22 = targets_fy22.dropna(subset=['FY22'])
targets_fy22 = targets_fy22[targets_fy22['FY22'] != 0]
targets_fy22['FY22'] = targets_fy22['FY22'] / 12
targets_fy22['FY'] = 2022
targets_fy22 = targets_fy22.rename(columns={'FY22': 'target_order_intake_amount_eur'})
num_fy22 = len(targets_fy22)

targets_fy23_pci = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY23_PCI', 'tm_check', 'pure tm']]
targets_fy23_pci = targets_fy23_pci.dropna(subset=['FY23_PCI'])
targets_fy23_pci = targets_fy23_pci[targets_fy23_pci['FY23_PCI'] != 0]
targets_fy23_pci['FY23_PCI'] = targets_fy23_pci['FY23_PCI'] / 12
targets_fy23_pci['FY'] = 2023
targets_fy23_pci = targets_fy23_pci.rename(columns={'FY23_PCI': 'target_order_intake_amount_eur'})
num_fy23_p = len(targets_fy23_pci)

targets_fy23_sys = targets[['customer_name', 'tier', 'tier_new', 'company_code_n', 'type', 'FY23_Systems', 'tm_check', 'pure tm']]
targets_fy23_sys = targets_fy23_sys.dropna(subset=['FY23_Systems'])
targets_fy23_sys = targets_fy23_sys[targets_fy23_sys['FY23_Systems'] != 0]
targets_fy23_sys['FY23_Systems'] = targets_fy23_sys['FY23_Systems'] / 12
targets_fy23_sys['FY'] = 2023
targets_fy23_sys = targets_fy23_sys.rename(columns={'FY23_Systems': 'target_order_intake_amount_eur'})
num_fy23_s = len(targets_fy23_sys)

In [5]:
fy_21 =  [202104, 202105, 202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203]
fy_22 =  [202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211, 202212, 202301, 202302, 202303]
fy_23 =  [202304, 202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403]

In [6]:
targets_fy21 = pd.concat([targets_fy21] * 12, ignore_index=True)
targets_fy21 = targets_fy21.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy21['year_month'] = fy_21 * num_fy21
targets_fy21['for_bu'] = 'PCI'
targets_fy21['tm_check'] = 'no'
targets_fy21['tm_check'] = 'no'

targets_fy22 = pd.concat([targets_fy22] * 12, ignore_index=True)
targets_fy22 = targets_fy22.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy22['year_month'] = fy_22 * num_fy22
targets_fy22['for_bu'] = 'PCI'

targets_fy23_pci = pd.concat([targets_fy23_pci] * 12, ignore_index=True)
targets_fy23_pci = targets_fy23_pci.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy23_pci['year_month'] = fy_23 * num_fy23_p
targets_fy23_pci['for_bu'] = 'PCI'

targets_fy23_sys = pd.concat([targets_fy23_sys] * 12, ignore_index=True)
targets_fy23_sys = targets_fy23_sys.sort_values(by=['customer_name', 'target_order_intake_amount_eur'])
targets_fy23_sys['year_month'] = fy_23 * num_fy23_s
targets_fy23_sys['for_bu'] = 'Systems&ADS'

In [7]:
combined_df = pd.concat([targets_fy21, targets_fy22, targets_fy23_pci, targets_fy23_sys], ignore_index=True)

combined_df['FY'] = (pd.to_datetime(combined_df['year_month'], format='%Y%m') - pd.DateOffset(months=3)).dt.strftime('%d-%m-%Y')

In [8]:
combined_df['customer_name'] = combined_df['customer_name'].str.strip()

In [9]:
def apply_shift(value):
    if value >= 4:
        return value - 3
    else:
        return value + 9
    
combined_df['year_month_copy'] = combined_df['year_month'].astype(str)
combined_df['m_num'] = combined_df['year_month_copy'].str[-2:].astype(int)
combined_df['m_num'] = combined_df['m_num'].apply(apply_shift)

In [10]:
combined_df = combined_df[['customer_name', 'type',
       'target_order_intake_amount_eur', 'tm_check', 'pure tm', 'FY',
       'year_month', 'for_bu', 'm_num']]

Sometimes types of partners are changing, it is necessary to change them to proper one

In [11]:
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [12]:
df_customers = df_customers[['sold_to_customer', 'company_code_n', 'customer_name','tier_new', 'type','countries']]
df_customers = df_customers.drop_duplicates(subset=['customer_name', 'type'], keep='first')

In [13]:
new_combined_df = combined_df.merge(df_customers, on=['customer_name', 'type'], how='left', suffixes=('', '_y'))

In [14]:
not_empty_df = new_combined_df[~new_combined_df['countries'].isna()]
empty_df = new_combined_df[new_combined_df['countries'].isna()]

In [15]:
columns = list(combined_df.columns)
empty_df = empty_df[columns]

In [16]:
empty_df = empty_df.merge(df_customers, on=['customer_name'], how='left', suffixes=('', '_y'))

columns = list(not_empty_df.columns)
empty_df = empty_df[columns]

In [17]:
final_combined_df = pd.concat([not_empty_df, empty_df], ignore_index=True)

In [18]:
final_combined_df['target_order_intake_amount_eur'].sum() == combined_df['target_order_intake_amount_eur'].sum()

True

In [19]:
writer = pd.ExcelWriter('data_files/outcome/target.xlsx')
# Save each DataFrame to a separate sheet in the same file
final_combined_df.to_excel(writer, sheet_name='target', index=False)
# Save the file
writer.close()

In [ ]:
# check some companies which are not in db yet Ase, rmo